In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
dy = pd.read_csv(
         '../../../AprOct15_trimmed.csv')

In [4]:
dn = pd.read_csv(
         '../../../Nov15_trimmed.csv')

In [5]:
targets = dn[dn['PROD_GROUP_CODE'] == 5]['INDIV_ID'].unique()
year_indivs = pd.DataFrame({'indiv':dy['INDIV_ID'].unique()})
year_indivs['label'] = np.where(year_indivs['indiv'].isin(targets), 1, 0) 

## Model Age
### Example of 2015 trimmed, Check the 0-1 proportion

In [6]:
dy['MODEL_AGE'] = pd.to_datetime(dy['DATE'].min()).year - dy['MODEL_YEAR']

In [7]:
less_than_15_modelage = dy.query('MODEL_AGE < 15')['VEHICLE_ID']

In [8]:
indiv_less_than_15_modelage = dy[dy['VEHICLE_ID'].isin(less_than_15_modelage)]['INDIV_ID']

In [9]:
year_indivs['indiv_less_than_15_modelage'] = np.where(year_indivs.indiv.isin(indiv_less_than_15_modelage), 1, 0)

In [10]:
year_indivs['indiv_less_than_15_modelage'].value_counts()

1    2293525
0     272040
Name: indiv_less_than_15_modelage, dtype: int64

#### So, we can edit the model year to change the 0-1 proportion

## MAKE
### Example of 2015 trimmed, Check the 0-1 proportion

In [11]:
cat = ['MAKE', 'MODEL', 'SUB_MODEL']
for i in cat:
    dq = dy[i].groupby([dy['TRAN_ID'], dy['DATE']]).count().to_frame()
    print(i,  'Data Miss Rate', len(dq[dq[i]==0]) / len(dq[i]))

MAKE Data Miss Rate 0.029364163881582542
MODEL Data Miss Rate 0.029364163881582542
SUB_MODEL Data Miss Rate 0.17267235630775785


In [12]:
for i in cat:
    T = dy[i].groupby([dy['TRAN_ID'], dy['DATE']]).value_counts().to_frame().rename(columns = {i:'count'})
    T.reset_index(inplace =True)
    T.drop('count', axis = 1, inplace = True)
    T = T.groupby(i).count()
    T.reset_index(inplace = True)
    T.drop('DATE', axis = 1, inplace = True)
    T.rename(columns = {'TRAN_ID':'total_trans'}, inplace = True)
    print(i, 'top 10 share in transactions:', T.sort_values('total_trans', ascending = False)[:10].sum()[1]/T['total_trans'].sum())

MAKE top 10 share in transactions: 0.7266505603262451
MODEL top 10 share in transactions: 0.11706154916957115
SUB_MODEL top 10 share in transactions: 0.508766788625779


#### Using 2015 is good

In [13]:
T = dy['MAKE'].groupby([dy['TRAN_ID'], dy['DATE']]).value_counts().to_frame().rename(columns = {'MAKE':'count'})
T.reset_index(inplace =True)
T.drop('count', axis = 1, inplace = True)
T = T.groupby('MAKE').count()
T.drop('DATE', axis = 1, inplace = True)
T.rename(columns = {'TRAN_ID':'total transactions'}, inplace = True)

In [14]:
top_10_make = T.sort_values('total transactions', ascending = False)[:10]

In [15]:
top_10_make.reset_index(inplace =True)
top_10_make = top_10_make['MAKE']

In [16]:
indiv_top_10_make = dy[dy['MAKE'].isin(top_10_make)]['INDIV_ID']

In [17]:
year_indivs['indiv_top_10_make'] = np.where(year_indivs.indiv.isin(indiv_top_10_make), 1, 0)

In [18]:
year_indivs['indiv_top_10_make'].value_counts()

1    1927812
0     637753
Name: indiv_top_10_make, dtype: int64

#### Also, we can edit the model year to change the 0-1 proportion